In [ ]:
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import optuna
import time
import re
import warnings

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [ ]:

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\vlade\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vlade\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vlade\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv("IMDB Dataset.csv", on_bad_lines='skip')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
data['sentiment'].unique()

array(['positive', 'negative'], dtype=object)

In [ ]:
def preprocess_text(text):
    # Удаление специальных символов и приведение к нижнему регистру
    text = re.sub(r'\W', ' ', text)
    #text = text.lower()

    # Токенизация
    tokens = nltk.word_tokenize(text)

    # Удаление стоп-слов
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Лемматизация
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

In [ ]:
data['sentiment'] = data['sentiment'].map({'positive': 1, 'negative': 0})

In [ ]:
data['sentiment']

0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64

In [ ]:
preprocessed_texts = [preprocess_text(text) for text in data['review']]

### TF-IDF

In [ ]:
# Векторизация с помощью TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preprocessed_texts)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, data['sentiment'], test_size=0.2, random_state=42)

In [ ]:
# Обучение модели логистической регрессии
clf = LogisticRegression(max_iter=100)
clf.fit(X_train, y_train)



# Предсказание меток для тестовых данных
y_pred = clf.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Вывод отчета о классификации и матрицы ошибок
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.90
              precision    recall  f1-score   support

           0       0.91      0.88      0.89      4961
           1       0.89      0.91      0.90      5039

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000

[[4369  592]
 [ 443 4596]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier


In [ ]:
model = RandomForestClassifier(n_estimators=1000,
                              criterion='log_loss',
                              max_depth=20,
                              min_samples_split=10,
                              min_samples_leaf=1,
                              random_state = 0)


model.fit(X_train, y_train)

y_pred = model.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print(f'Accuracy(train) = {accuracy}')

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy(test) = {accuracy}')

Accuracy(train) = 0.91985
Accuracy(test) = 0.8511


In [ ]:
def objective(trial):
    start_time = time.time()
    # Определяем гиперпараметры, которые будет оптимизировать Optuna
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 5, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss'])

    # Создаем модель RandomForestClassifier с подобранными параметрами
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=0
    )

    # Обучаем модель
    model.fit(X_train, y_train)

    # Предсказываем и оцениваем точность на тестовой выборке
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    end_time = time.time()
    print(f'Время: {(end_time - start_time)/60:.2f} минут')

    return accuracy

# Создаем объект Optuna и запускаем оптимизацию
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)



[I 2024-08-22 18:50:33,422] A new study created in memory with name: no-name-2e362d7b-6ff7-442f-bfe3-817d57a1b136
[I 2024-08-22 18:52:55,706] Trial 0 finished with value: 0.8532 and parameters: {'n_estimators': 584, 'max_depth': 26, 'min_samples_split': 19, 'min_samples_leaf': 9, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8532.


Время: 2.37 минут


[I 2024-08-22 18:55:00,755] Trial 1 finished with value: 0.8535 and parameters: {'n_estimators': 507, 'max_depth': 28, 'min_samples_split': 5, 'min_samples_leaf': 8, 'criterion': 'gini'}. Best is trial 1 with value: 0.8535.


Время: 2.08 минут


[I 2024-08-22 18:57:16,792] Trial 2 finished with value: 0.8523 and parameters: {'n_estimators': 563, 'max_depth': 27, 'min_samples_split': 3, 'min_samples_leaf': 9, 'criterion': 'log_loss'}. Best is trial 1 with value: 0.8535.


...

Время: 5.04 минут


[I 2024-08-22 21:51:21,954] Trial 48 finished with value: 0.8566 and parameters: {'n_estimators': 434, 'max_depth': 29, 'min_samples_split': 17, 'min_samples_leaf': 5, 'criterion': 'log_loss'}. Best is trial 13 with value: 0.8612.


Время: 1.97 минут


[I 2024-08-22 21:53:40,939] Trial 49 finished with value: 0.8554 and parameters: {'n_estimators': 598, 'max_depth': 22, 'min_samples_split': 10, 'min_samples_leaf': 2, 'criterion': 'entropy'}. Best is trial 13 with value: 0.8612.


Время: 2.32 минут


In [ ]:
print("Best trial:")
trial = study.best_trial

print(f"  Accuracy: {trial.value}")
print("  Best hyperparameters: ", trial.params)

best_model = RandomForestClassifier(**trial.params, random_state=0)
best_model.fit(X_train, y_train)

y_train_pred = best_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f'Accuracy(train) = {train_accuracy}')

y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'Accuracy(test) = {test_accuracy}')

Best trial:
  Accuracy: 0.8612
  Best hyperparameters:  {'n_estimators': 983, 'max_depth': 30, 'min_samples_split': 17, 'min_samples_leaf': 1, 'criterion': 'log_loss'}
Accuracy(train) = 0.94625
Accuracy(test) = 0.8612


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [ ]:
svc = SVC()
start_time = time.time()
# Определение параметров для поиска
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print("Лучшие параметры:", grid_search.best_params_)
print("Наилучший результат:", grid_search.best_score_)

best_svc = grid_search.best_estimator_

y_pred = best_svc.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

end_time = time.time()
print(f'Время: {(end_time - start_time)/60:.2f} минут')

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Лучшие параметры: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Наилучший результат: 0.8967999248728781
Accuracy: 0.90
              precision    recall  f1-score   support

           0       0.91      0.89      0.90      4961
           1       0.89      0.92      0.91      5039

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000

[[4416  545]
 [ 416 4623]]
Время: 150.49 минут


### CountVectorizer

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(preprocessed_texts)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, data['sentiment'], test_size=0.2, random_state=42)

In [ ]:
# Обучение модели логистической регрессии
clf = LogisticRegression(max_iter=100)
clf.fit(X_train, y_train)



# Предсказание меток для тестовых данных
y_pred = clf.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Вывод отчета о классификации и матрицы ошибок
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.88
              precision    recall  f1-score   support

           0       0.89      0.88      0.88      4961
           1       0.88      0.89      0.89      5039

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000

[[4359  602]
 [ 557 4482]]


In [ ]:
model = RandomForestClassifier(n_estimators=1000,
                              criterion='log_loss',
                              max_depth=20,
                              min_samples_split=10,
                              min_samples_leaf=1,
                              random_state = 0)


model.fit(X_train, y_train)

y_pred = model.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print(f'Accuracy(train) = {accuracy}')

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy(test) = {accuracy}')

Accuracy(train) = 0.911275
Accuracy(test) = 0.8641


In [ ]:
def objective(trial):
    start_time = time.time()
    # Определяем гиперпараметры, которые будет оптимизировать Optuna
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 5, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss'])

    # Создаем модель RandomForestClassifier с подобранными параметрами
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=0
    )

    # Обучаем модель
    model.fit(X_train, y_train)

    # Предсказываем и оцениваем точность на тестовой выборке
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    end_time = time.time()
    print(f'Время: {(end_time - start_time)/60:.2f} минут')

    return accuracy

# Создаем объект Optuna и запускаем оптимизацию
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15)



[I 2024-08-23 11:23:26,429] A new study created in memory with name: no-name-4e09598f-3d9c-4c13-b3b7-312d4af330b4
[I 2024-08-23 11:26:16,477] Trial 0 finished with value: 0.8613 and parameters: {'n_estimators': 897, 'max_depth': 19, 'min_samples_split': 20, 'min_samples_leaf': 1, 'criterion': 'gini'}. Best is trial 0 with value: 0.8613.


Время: 2.83 минут


[I 2024-08-23 11:27:57,995] Trial 1 finished with value: 0.8582 and parameters: {'n_estimators': 644, 'max_depth': 18, 'min_samples_split': 17, 'min_samples_leaf': 7, 'criterion': 'log_loss'}. Best is trial 0 with value: 0.8613.


Время: 1.69 минут


[I 2024-08-23 11:29:51,229] Trial 2 finished with value: 0.8604 and parameters: {'n_estimators': 578, 'max_depth': 24, 'min_samples_split': 4, 'min_samples_leaf': 9, 'criterion': 'gini'}. Best is trial 0 with value: 0.8613.


...

Время: 4.17 минут


[I 2024-08-23 11:53:28,704] Trial 13 finished with value: 0.8653 and parameters: {'n_estimators': 791, 'max_depth': 29, 'min_samples_split': 13, 'min_samples_leaf': 2, 'criterion': 'entropy'}. Best is trial 11 with value: 0.8683.


Время: 3.84 минут


[I 2024-08-23 11:55:30,645] Trial 14 finished with value: 0.8575 and parameters: {'n_estimators': 997, 'max_depth': 14, 'min_samples_split': 15, 'min_samples_leaf': 4, 'criterion': 'entropy'}. Best is trial 11 with value: 0.8683.


Время: 2.03 минут


In [ ]:
print("Best trial:")
trial = study.best_trial

print(f"  Accuracy: {trial.value}")
print("  Best hyperparameters: ", trial.params)

best_model = RandomForestClassifier(**trial.params, random_state=0)
best_model.fit(X_train, y_train)

y_train_pred = best_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f'Accuracy(train) = {train_accuracy}')

y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'Accuracy(test) = {test_accuracy}')

Best trial:
  Accuracy: 0.8683
  Best hyperparameters:  {'n_estimators': 985, 'max_depth': 30, 'min_samples_split': 14, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Accuracy(train) = 0.9367
Accuracy(test) = 0.8683
